### Ejercicio 1

Desarrollar un notebook de Jupyter, denominado “tweetCount.ipynb” en el que se utilice como fuente de datos Kafka, y en concreto el topic kafkaTwitter:

- se establezca una duración de batch de un segundo
- se muestre, cada 5 segundos
- el número de tweets recibidos en los últimos 10 segundos.

In [58]:
import findspark
import requests
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from pyspark.streaming.kafka import KafkaUtils

In [59]:
# ubicación de spark
findspark.init('/home/j/spark-2.4.7-bin-hadoop2.7')

In [61]:
# necesario para cargar la librería de spark streaming y kafka
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 pyspark-shell'

In [63]:
# nombre del topic
topic = 'kafkaTwitter'

In [64]:
# creación de los contextos de spark
sc = SparkContext("local[*]")
ssc = StreamingContext(sc, 1)

In [65]:
# creación del DStream a partir de Kafka
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'localhost:9092, localhost:9093',
                        'group.id':'test-group'})

In [68]:
# ventana de los tweets definida tal cual se pide en el ejercicio:
# - tweets recibidos en los últimos 10 segundos
# - muetra cada 5 segundos
tweets = kafkaStream.window(10,5)

In [69]:
# impresión de los batches tal cual están configurados
tweets.pprint()

In [70]:
# iniciado del stream
ssc.start()

# nota un log de output para que se vea como funciona

-------------------------------------------
Time: 2020-12-22 13:46:03
-------------------------------------------
(None, 'Seguridad Apple - El phishing de Apple ya es detectado por Google Chrome y el dominio tumbado http://t.co/G3nO51K7FX #Apple #phishing')
(None, 'Me da #vergüenza el #debate entre @Canete2014_ y @ElenaValenciano. Basta ya de este tipo de política.')
(None, 'HP estaría preparando un portátil Windows de solament...http://t.co/DvQjGepeAd')
(None, 'Nuestro Secretario Político Íñigo Errejón (@ierrejon) estará esta mañana en @DebatAlRojoVivo. ¡No te lo pierdas! http://t.co/P2ZvYZgWUD')
(None, 'Alguien quiere hablar en wpp? Pidemelo :D😘 - Ahora no tengo ganas, me voy a dormir y in...   http://t.co/dIEPCPTD6F')
(None, 'Audi crea el combustible del futuro con aire y agua  http://t.co/NFTgHz7Uq8 http://t.co/dBhHlusNDZ')
(None, 'Sony Xperia M4 Aqua ya puede ser tuyo por 299 euros - http://t.co/mXwfnx8tPt http://t.co/OZvPUjoP9A')
(None, 'Más leída 24h: Viñeta publicada en "de Vol

In [71]:
ssc.stop()